<img src = "../../Data/bgsedsc_0.jpg">

# Extended Project: KNN project

The aim of this project is to predict the probability of death of a patient that is entering an ICU (Intensive Care Unit). The dataset comes from MIMIC project (https://mimic.physionet.org/) which refers to the Medical Information Mart for Intensive Care. The preprocessing of the data is the same for both the KNN and SVM projects, with the difference that SVM tackle class imbalance.

## Modules and auxiliar functions import

The relevant modules are loaded and the auxiliar functions from *helper_functions* are imported. It is also set a random seed to ensure reproducibility and it is checked the current working directory.

In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(os.path.dirname(currentdir))
sys.path.insert(1, parentdir)
from utils.helper_functions import *

import pandas as pd
import seaborn as sns
import numpy as np
import sklearn
import ipywidgets

from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, HalvingGridSearchCV
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.impute import KNNImputer
from category_encoders import TargetEncoder
from dateutil.relativedelta import relativedelta
from scipy.stats import skew
from collections import Counter
from imblearn.under_sampling import AllKNN

import warnings
warnings.filterwarnings('ignore')

np.random.seed(3123)

!pwd

ModuleNotFoundError: No module named 'utils'

## Data load

The **training dataset** is imported as a pandas DataFrame and the first 5 rows are displayed to get an overview of the structure and the contents. The dataset consists of 20885 observations and 44 columns.

In [ ]:
import pandas as pd

data=pd.read_csv('/Users/bertacanal/Desktop/cml23-probability-of-death-with-k-nn/mimic_train.csv')
print(data.shape)
data.head()

The **test dataset** is imported as a pandas DataFrame and the first 5 rows are displayed to get an overview of the structure and the contents. This dataset will be used to produce predictions and consists of 5221 observations and 39 columns. This dataset does not include the variable *'HOSPITAL_EXPIRE_FLAG'*, therefore, a copy of this dataset is renamed as *X_test*.

In [ ]:
data_test=pd.read_csv('/Users/bertacanal/Desktop/cml23-probability-of-death-with-k-nn/mimic_test_death.csv')
X_test = data_test.copy()
print(X_test.shape)
X_test.sort_values('icustay_id').head()

# Preprocessing of the data

## Exploration of the dataset

Each observation of the training dataset correspond to the ICU stay of a patient. The outcome to predict is the column HOSPITAL_EXPIRE_FLAG, which indicates if the patient died during the current hospital stay (= 1) or survived (= 0). There are ID columns related to the ICU stay (*hadm_id* and *icustay_id*) and to the patient (*subject_id*). Other columns refer to the vitals of the patient (when entering the ICU) and general information. The specific meaning of each feature is specified in the table below (extracted from the file *mimic_patient_metadata.xlsx*). 

In [ ]:
meaning=pd.read_excel('/Users/bertacanal/Desktop/cml23-probability-of-death-with-k-nn/mimic_patient_metadata.xlsx', header = 4)
pd.set_option('display.max_colwidth', None)
meaning

The following bar plot represents the distribution of the *'HOSPITAL_EXPIRE_FLAG'* variable, showing the counts of *Survived* and *Not Survived* categories.

In [ ]:
labels = {0: 'Survived', 1: 'Not Survived'}
value_counts = data['HOSPITAL_EXPIRE_FLAG'].map(labels).value_counts()

total_patients = value_counts.sum()
plot = value_counts.plot(kind='bar', rot=0)

for index, value in enumerate(value_counts):
    percentage = (value / total_patients) * 100
    plt.text(index, value, f"{value} ({percentage:.2f}%)", ha='center', va='bottom')

plt.ylabel('Number of patients')
plt.title('Variable HOSPITAL_EXPIRE_FLAG')

plt.show()

## Addition of metadata

As shown in the description of the variables, the *ICD9_diagnosis* column includes the main cause/disease of patient condition. However, a patient can have co-ocurrent diseases (comorbodities). The file containing the secondary codes is *MIMIC_diagnoses.csv* and some merge is required to consider these additional diagnoses. It is assumed that all of the comorbidities were diagnosed on the first day of the patient in the ICU.

In [ ]:
MIMIC_diagnoses = pd.read_csv('/Users/bertacanal/Desktop/cml23-probability-of-death-with-k-nn/extra_data/MIMIC_diagnoses.csv')
MIMIC_diagnoses.rename(columns={'HADM_ID': 'hadm_id'}, inplace=True)
MIMIC_diagnoses.head()

A reasonable threshold of diagnoses to consider for the project is set to 20, but given that the training dataset already includes the main one, the merge will only include those ICD9_CODES assigned to SEQ_NUM from 2 to 20. For the missing data it is created a missing class called *MISSING_DIAGNOSIS*. Both dataframes are join on 'hadm_id' (which refers to the hospital stay identifier) because the *MIMIC_diagnoses.csv* includes information about different hospital stays of the different of each patients.

In [ ]:
for seq_num in range (2, 21):
    secondary_codes = MIMIC_diagnoses[MIMIC_diagnoses['SEQ_NUM'] == seq_num].copy()
    secondary_codes.head()

    secondary_codes[f'ICD9_CODE_{seq_num}'] = secondary_codes['ICD9_CODE']
    secondary_codes.head()
    data = pd.merge(data,secondary_codes[['hadm_id',f'ICD9_CODE_{seq_num}']],on='hadm_id', how='left')
    data[f'ICD9_CODE_{seq_num}'].fillna('MISSING_DIAGNOSIS', inplace = True)
    X_test = pd.merge(X_test,secondary_codes[['hadm_id',f'ICD9_CODE_{seq_num}']],on='hadm_id', how='left')
    X_test[f'ICD9_CODE_{seq_num}'].fillna('MISSING_DIAGNOSIS', inplace = True)
    
data.head()

In [ ]:
proportions = []

for seq_num in range (2, 21):
    column_name = f'ICD9_CODE_{seq_num}'
    if column_name in data.columns:
        counts = data[column_name].value_counts()
        if 'MISSING_DIAGNOSIS' in counts:
            proportion = round((counts['MISSING_DIAGNOSIS'] / data.shape[0])*100,2)
            proportions.append((column_name, proportion))
proportions_df = pd.DataFrame(proportions, columns=['Column Name', 'Proportion of "MISSING_DIAGNOSIS" (%)'])
proportions_df

Since the proportion of missing data for those columns referring to a higher sequence number is high, it seems reasonable to drop some of those columns. Therefore, this project will set as threshold a proportion of 40% of missing data for the secondary codes and thus only consider until the column referring to the sequence number 12.

In [ ]:
data = data.drop(['ICD9_CODE_13', 'ICD9_CODE_14', 'ICD9_CODE_15', 'ICD9_CODE_16', 'ICD9_CODE_17', 'ICD9_CODE_18', 'ICD9_CODE_19', 'ICD9_CODE_20'], axis=1)
X_test = X_test.drop(['ICD9_CODE_13', 'ICD9_CODE_14', 'ICD9_CODE_15', 'ICD9_CODE_16', 'ICD9_CODE_17', 'ICD9_CODE_18', 'ICD9_CODE_19', 'ICD9_CODE_20'], axis=1)

## Dicernment between *X_train* and *y_train*

From the training dataset it should be distinguished between *X_train* (including all the features) and *y_train* (including the variable *HOSPITAL_EXPIRE_FLAG*).

In [ ]:
X_train = data.drop(['HOSPITAL_EXPIRE_FLAG'], axis=1)
y_train = data['HOSPITAL_EXPIRE_FLAG']

X_train.head()

The following code aims to compare the dimensions from *X_train* and *X_test*. As expected, the number of observations of the training dataset is higher than the testing dataset since we want to feed the model with as much data as possible to find and learn meaningful patterns.

In [ ]:
dimensions = [X_train.shape, X_test.shape]
dimensions_df = pd.DataFrame([(int(dim[0]), int(dim[1])) for dim in dimensions], index= ["X_train", "X_test"], columns=["Rows", "Columns"])
dimensions_df

To know which columns appear in *X_train* but not in *X_test* it is done a set difference operation. It is useful to know the columns in each dataset for the step related to drop the features not used in the model. As seen in the output, the columns that only appear in *X_train* correspond to *'DEATHTIME'*, *'DISCHTIME'*, *'DOD'* and *'LOS'*.

In [ ]:
columns_X_train = set(X_train.columns)
columns_X_test = set(X_test.columns)

columns_only_in_X_train = columns_X_train - columns_X_test
columns_only_in_X_train

## Creation of the *AGE* variable

The age could have an impact on the probability of death of a patient that is entering an ICU, however, there is no specific feature in the dataset referring to the age. An alternative could be to subtract the *DOB* (Date of Birth) from the *DOD* (Date of Death) but it has to be disregarded since the *DOD* is not known in the first day of a patient in an ICU and moreover it does not allow to compute the age of the patients who survive.

Therefore, the procedure followed consists on subtracting the *DOB* (Date of Birth) from the *ADMITTIME* (Admision datetime) since this results on the age of the patient in the admission day. These two date columns are converted to datatime objects and then the age is computed using the *relativedelta* function.

#### In the training dataset

In [ ]:
X_train['DOB'] = pd.to_datetime(X_train['DOB'], format='%Y-%m-%d %H:%M:%S')
X_train['ADMITTIME'] = pd.to_datetime(X_train['ADMITTIME'], format='%Y-%m-%d %H:%M:%S')

X_train['AGE'] = X_train.apply(lambda row: relativedelta(row['ADMITTIME'], row['DOB']).years, axis=1)

X_train['AGE'].describe()

#### In the testing dataset

In [ ]:
X_test['DOB'] = pd.to_datetime(X_test['DOB'], format='%Y-%m-%d %H:%M:%S')
X_test['ADMITTIME'] = pd.to_datetime(X_test['ADMITTIME'], format='%Y-%m-%d %H:%M:%S')

X_test['AGE'] = X_test.apply(lambda row: relativedelta(row['ADMITTIME'], row['DOB']).years, axis=1)

X_test['AGE'].describe()

#### Adjustments to the variable *AGE*

The quartiles and the mean seem to be reasonable. However, since the maximum value of the column *AGE* corresponds to 310 in both the training and testing data, an adjustment is needed.  The problem is that the age of patients older than 89 has been fixed to 300 at their first admission (Source: https://mimic.mit.edu/docs/iii/about/time/). 

In [ ]:
print('Patients older than 89:')
print(X_train[X_train['AGE'] > 89]['AGE'])

Therefore, it is reasonable to readjust the *AGE* by assigning 89 years as the maximum patient's age:

In [ ]:
X_train['AGE'] = X_train['AGE'].apply(lambda age: min(age, 89))
X_test['AGE'] = X_test['AGE'].apply(lambda age: min(age, 89))

## Features selection

Vital signs are considered important in medical care and disease diagnosis. They include heart rate, blood pressure, respiratory rate and body temperature. For example, a significant increase or decrease in heart rate may be an indicator of heart problems, high blood pressure can increase the risk of cardiovascular diseases, changes in respiratory rate can indicate respiratory or cardiovascular problems and a significant increase or decrease in temperature can be a sign of infection. Therefore, features related to the minimum, maximum and mean of the vital signs are included in the model.

Both the main disease and comorbidities play significant roles in determining a patient's probability of death. The main disease's severity directly influences the risk of mortality. Comorbidities, on the other hand, add complexity to the patient's health profile: they can complicate treatment, reduce the body's ability to respond to stress and weaken the immune system. Therefore, features related to the main disease and comorbodities are included in the model.

Both gender and age can impact a patient's probability of survival in the ICU. Gender can influence because of differences in biological, hormonal, and treatment responses and age affects physiological reserves and vulnerability to illness.

Patients admitted to the ICU in an emergency, such as after a sudden cardiac arrest or trauma, may have a higher initial severity of illness. This can increase the risk of mortality, therefore it seems reasonable that admission type infers on the probability of death. The first care unit to which a patient is assigned can provide valuable insights into the initial assessment of the patient's condition and potential factors that may influence their probability of death. At the same time, the type of insurance also infers because it affects the access to care and the care quality, since patients with certain insurance plans may receive care at facilities with higher ICU quality ratings, potentially leading to better outcomes. Marital status can also be an influenting factor through factors such as social support, advocacy and emotional well-being. For example, married patients may benefit from spousal support or better mental health. 

In many healthcare systems, medical care and treatment standards are designed to be uniform and evidence-based, focusing on clinical needs rather than ethnic or religious considerations. Therefore, ethnicity and religion are not included in the model. ID variables are also excluded since these are not predictive of death and add noise into the model, thus decreasing its performance. 

#### From the training dataset

The features not included in the model are dropped. From the training dataset we exclude the ID variables (*subject_id*, *hadm_id* and *icustay_id*), the features we don't know the first day of a patient in an ICU (*DOD*, *DISCHTIME*, *DEATHTIME* and *LOS*) and the variables we consider not to infer in the probability of death of a patient (*ETHNICITY*, *RELIGION* and *Diff*). In addition, the variables used to create the *AGE* variable, in particular *DOB* and *ADMITTIME*, are also excluded since its information is already contained by the model with the created *AGE* variable.

In [ ]:
X_train = X_train.drop(['subject_id', 'hadm_id', 'icustay_id', 'DOB', 'DOD', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'Diff', 'LOS', 'ETHNICITY', 'RELIGION'], axis=1)
print(X_train.shape)
X_train.head()

#### From the testing dataset

The features not included in the model are dropped. From the testing dataset we exclude the ID variables (*subject_id*, *hadm_id* and *icustay_id*) and the variables we consider not to infer in the probability of death of a patient (*ETHNICITY*, *RELIGION* and *Diff*). In addition, the variables used to create the *AGE* variable, in particular *DOB* and *ADMITTIME*, are also excluded since its information is already contained by the model with the created *AGE* variable.

As seen before, *X_test* does not contain the columns *'DEATHTIME'*, *'DISCHTIME'*, *'DOD'* and *'LOS'*. Therefore, these columns should not be droped from the testing dataset.

In [ ]:
X_test = X_test.drop(['subject_id', 'hadm_id', 'icustay_id', 'DOB','ADMITTIME', 'Diff', 'ETHNICITY', 'RELIGION'], axis=1)
print(X_test.shape)
X_test.head()

After dropping the variables, it can be observed that the number of columns on both *X_train* and *X_test* is equal:

In [ ]:
dimensions = [X_train.shape, X_test.shape]
dimensions_df = pd.DataFrame([(int(dim[0]), int(dim[1])) for dim in dimensions], index= ["X_train", "X_test"], columns=["Rows", "Columns"])
dimensions_df

The following code creates a list of the names of the categorical and numerical columns:

In [ ]:
categorical_features_names = ['GENDER', 'INSURANCE', 'ADMISSION_TYPE', 'MARITAL_STATUS', 'FIRST_CAREUNIT', 'ICD9_diagnosis', 'DIAGNOSIS', 'ICD9_CODE_2', 'ICD9_CODE_3', 'ICD9_CODE_4', 'ICD9_CODE_5', 'ICD9_CODE_6', 'ICD9_CODE_7', 'ICD9_CODE_8', 'ICD9_CODE_9', 'ICD9_CODE_10', 'ICD9_CODE_11', 'ICD9_CODE_12']

numerical_features_names = [name for name in X_train.columns if name not in categorical_features_names]

## Cardinality of categorical features

Categorical features will be treated differently depending on the cardinality. The following output provides information about the categorical features: *count* corresponds to the number of non-null values, *unique* indicates the number of unique categories in the feature, *top* refers to the most frequent category and *freq* corresponds to the frequency in which the top category occur.

In [ ]:
categorical_summary = []

for feature in categorical_features_names:
    summary = X_train[feature].describe()
    categorical_summary.append(summary)

categorical_summary_table = pd.DataFrame(categorical_summary, index=categorical_features_names)
categorical_summary_table

As can be seen in the output of the previous code, the low cardinality variables consist on *GENDER*, *INSURANCE*, *ADMISSION_TYPE*, *MARITAL_STATUS* and *FIRST_CAREUNIT* while the variables with high cardinality consists on the *DIAGNOSIS* feature and the variables including ICD9 codes.

The output is also useful to identify that most of the patients from the training dataset are males, the most frequent marital status is to be married, the insurance is Medicare, the admission type is emergency and the first ICU assigned is Medical Intensive Care Unit (MICU). 

Regarding the diagnoses, the most frequent main disease corresponds to *Coronary atherosclerosis of native coronary artery*. The most frequent comorbidities are *Acute kidney failure, unspecified* and *Unspecified essential hypertension*. This later information has been extracted from the file *MIMIC_metadata_diagnose* which includes the meaning of the ICD9 codes.

In [ ]:
meaning_diagnoses = pd.read_csv('/Users/bertacanal/Desktop/cml23-probability-of-death-with-k-nn/extra_data/MIMIC_metadata_diagnose.csv')
target_ICD9_CODES = ['41401', '5849', '4019']
meaning_diagnoses[meaning_diagnoses['ICD9_CODE'].isin(target_ICD9_CODES)]

## Treatment of the low cardinality categorical features

Focusing on the low cardinality categorical features, let's see more in detail how many observations are included in each category.

In [ ]:
pd.DataFrame(X_train['GENDER'].value_counts())

In [ ]:
pd.DataFrame(X_train['FIRST_CAREUNIT'].value_counts())

In [ ]:
pd.DataFrame(X_train['INSURANCE'].value_counts())

In [ ]:
pd.DataFrame(X_train['ADMISSION_TYPE'].value_counts())

In [ ]:
pd.DataFrame(X_train['MARITAL_STATUS'].value_counts())

Given that *MARITAL_STATUS* has categories with a low number of observations, it is reasonable to group the categories that includes less than 5% of the total observations in *X_train*:

In [ ]:
marital_status_counts = X_train['MARITAL_STATUS'].value_counts()
categories_to_replace = marital_status_counts[marital_status_counts < 0.05 * len(X_train)].index

X_train.loc[X_train['MARITAL_STATUS'].isin(categories_to_replace), 'MARITAL_STATUS'] = 'OTHER'
X_test.loc[X_test['MARITAL_STATUS'].isin(categories_to_replace), 'MARITAL_STATUS'] = 'OTHER'

As can be seen, this preprocessing step group the categories *'SEPARATED'*, *'UNKNOWN (DEFAULT)'* and *'LIFE PARTNER'* with the category *'OTHERS'*:

In [ ]:
pd.DataFrame(X_train['MARITAL_STATUS'].value_counts())

The distribution of the low cardinality categorical features are depicted in the following graphs:

In [ ]:
low_cardinality_categorical_names = ['GENDER', 'INSURANCE', 'ADMISSION_TYPE', 'MARITAL_STATUS', 'FIRST_CAREUNIT']

fcols = 3
frows = (len(low_cardinality_categorical_names) + fcols - 1) // fcols
plt.figure(figsize=(15, 4 * frows))

for i, col in enumerate(low_cardinality_categorical_names):
    plt.subplot(frows, fcols, i + 1)
    sns.countplot(data=X_train, x=col)
    plt.title(f'Distribution of {col}')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## Handle missing data

Missing data will be handled differently depending on the type of feature: low cardinality categorical features, high cardinality categorical features and numerical features.

### Low cardinality categorical features

The first step is to check the null values per feature for both the X_train and X_test:

In [ ]:
X_train_low_cardinality = X_train[low_cardinality_categorical_names]
X_train_low_cardinality.isnull().sum()

In [ ]:
X_test_low_cardinality = X_test[low_cardinality_categorical_names]
X_test_low_cardinality.isnull().sum()

The previous output indicates that the only low cardinality categorical feature with missing data corresponds to *MARITAL_STATUS*. To handle missing data from this feature, the *SimpleImputer* will be used to replace missing values by the most frequent value of this column. To check that this step has been done correctly, we can see that there is no missing data in the low cardinality categorical features from both datasets after the imputation.

In [ ]:
imp_frequent = SimpleImputer(missing_values = np.nan, strategy= 'most_frequent')

imp_frequent.fit(X_train_low_cardinality)

X_train_categorical_transform = pd.DataFrame(imp_frequent.transform(X_train_low_cardinality), columns = low_cardinality_categorical_names)
X_test_categorical_transform = pd.DataFrame(imp_frequent.transform(X_test_low_cardinality), columns = low_cardinality_categorical_names)

X_train[low_cardinality_categorical_names] = X_train_categorical_transform
X_test[low_cardinality_categorical_names] = X_test_categorical_transform

print("Missing data in low cardinality categorical features from X_train: ", X_train[low_cardinality_categorical_names].isnull().sum().sum())
print("Missing data in low cardinality categorical features from X_test: ", X_test[low_cardinality_categorical_names].isnull().sum().sum())

X_train[low_cardinality_categorical_names].head()

### High cardinality categorical features

The first step is to check the null values per feature for both the X_train and X_test. Since it was created a missing class for the missing data from the secondary codes, there aren't null values present in the high cardinality categorical features.

In [ ]:
high_cardinality_categorical_names = ['ICD9_diagnosis', 'DIAGNOSIS', 'ICD9_CODE_2', 'ICD9_CODE_3', 'ICD9_CODE_4', 'ICD9_CODE_5', 'ICD9_CODE_6', 'ICD9_CODE_7', 'ICD9_CODE_8', 'ICD9_CODE_9', 'ICD9_CODE_10', 'ICD9_CODE_11', 'ICD9_CODE_12']

In [ ]:
X_train_high_cardinality = X_train[high_cardinality_categorical_names]
X_train_high_cardinality.isnull().sum()

In [ ]:
X_test_high_cardinality = X_test[high_cardinality_categorical_names]
X_test_high_cardinality.isnull().sum()

### Numerical features

As in the previous cases, let's first check the missing data per numerical feature in both X_train and X_test. All numerical features has missing data except for the variable *AGE*.

In [ ]:
X_train_numerical_columns = X_train[numerical_features_names]
X_train_numerical_columns.isnull().sum().sort_values(ascending=False)

In [ ]:
X_test_numerical_columns = X_test[numerical_features_names]
X_test_numerical_columns.isnull().sum().sort_values(ascending=False)

The first step to deal with missing data from numerical features will consists on removing columns with a considerable high proportion of missing data. In this case it is established a percentage of 20% as the percentage of non-null answers required to keep a column. As can be seen, the dimensions before and after computing this step are the same, therefore it has not removed any column.

In [ ]:
initial_columns = X_train_numerical_columns

print(X_train.shape)
X_train = X_train.dropna(axis=1, thresh=round(0.2 * len(X_train.index)))
print(X_train.shape)

dropped_columns = list(set(initial_columns) - set(X_train.columns))
X_test = X_test.drop(columns=dropped_columns)

The missing data from the vitals of the patient (when entering the ICU) can be imputed by the mean. A possible consideration could be to impute group by gender, therefore using the mean of the corresponding gender to impute the vitals. This option was disregarded since the means of the vitals do not vary much between females and males.

In [ ]:
X_train[["HeartRate_Mean", "SysBP_Mean", "DiasBP_Mean", "MeanBP_Mean", "RespRate_Mean", "TempC_Mean", "SpO2_Mean", "Glucose_Mean", "GENDER"]].groupby("GENDER").mean()

Since it seems not reasonable to use stratified average statistics to impute the vitals, it will be proceed with the *KNNImputer* to deal with missing data using k-Nearest Neighbors. Therefore, the missing data from the vitals will be imputed using the mean value from n_neighbors nearest neighbors found in the training dataset. The parameter *weights* is set to *'distance'* to weight points by the inverse of their distance and *n_neighbors* is set to 5. To check that this step has been done correctly, we can see that there is no missing data in the numerical features from both datasets after the imputation.

In [ ]:
imp_knn = KNNImputer(n_neighbors= 5, weights = 'distance')

imp_knn.fit(X_train[numerical_features_names])

X_train[numerical_features_names] = pd.DataFrame(imp_knn.transform(X_train[numerical_features_names]), columns = numerical_features_names)
X_test[numerical_features_names] = pd.DataFrame(imp_knn.transform(X_test[numerical_features_names]), columns = numerical_features_names)

print("Missing data in numerical features from X_train: ", X_train[numerical_features_names].isnull().sum().sum())
print("Missing data in numerical features from X_test: ", X_test[numerical_features_names].isnull().sum().sum())

X_train[numerical_features_names].head()

## Check skewness

The skewness is checked for the numerical features. The feature is considered slightly skewed if the skewness is between -1 and -0.5 (negative skewed) or between 0.5 and 1 (positive skewed). The feature is considered extremely skewed if the skewness is lower than -1 (negative skewed) or greater than 1 (positive skewed). 

In [ ]:
skewness_results = {}
num_features = len(numerical_features_names)
num_plots_per_row = 4
num_rows = (num_features + num_plots_per_row - 1) // num_plots_per_row

# Create a figure and an array of subplots
fig, axes = plt.subplots(num_rows, num_plots_per_row, figsize=(16, 4 * num_rows))

# Loop through each numerical feature
for i, col in enumerate(numerical_features_names):
    col_name = col
    col_skew = skew(X_train[col])
    col_skew_rounded = round(col_skew, 4)
    skewness_results[col_name] = col_skew_rounded
    row_idx = i // num_plots_per_row
    col_idx = i % num_plots_per_row
    
    # Plot a histogram in the current subplot
    sns.histplot(X_train[col], kde=True, ax=axes[row_idx, col_idx])
    axes[row_idx, col_idx].set_title(f'Skewness for {col_name}: {col_skew_rounded}')
    
# Remove empty subplots
for i in range(num_features, num_rows * num_plots_per_row):
    row_idx = i // num_plots_per_row
    col_idx = i % num_plots_per_row
    fig.delaxes(axes[row_idx, col_idx])

# Adjust layout spacing
plt.tight_layout()

# Show all the plots
plt.show()

A log transformation is applied to handle extremely right-skewed features. Applying this transformtaion is useful for the skewed data since the distribution resembles more a normal. It is also useful to decrease the effect of outliers, thus obtaining a more robust model. As shown in the output, the log transformed numerical features log transformed correspond to *'DiasBP_Max'*, *'MeanBP_Max'*, *'RespRate_Max'*, *'Glucose_Min'*, *'Glucose_Max'* and *'Glucose_Mean'*.

In [ ]:
variables_to_transform = []

for col in numerical_features_names:
    col_skew = skew(X_train[col])
    if col_skew > 1:
        variables_to_transform.append(col)

for col in variables_to_transform:
    X_train[col] = np.log(X_train[col])
    X_test[col] = np.log(X_test[col])

print("Numerical features log transformed:", variables_to_transform)

## Correlation among numeric features

The correlation among numeric features is computed since highly correlated features do not bring a considerable additional information, thus leading to collinearity issues:

In [ ]:
correlation_matrix = X_train[numerical_features_names].corr()
correlation_matrix

Some sources set a Pearson's correlation coefficient of 0.80 as the threshold to consider two variables highly correlated. Therefore, the following output aims to find pairs of features with correlations higher than the threshold.

In [ ]:
threshold = 0.80

correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i + 1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            feature1 = correlation_matrix.columns[i]
            feature2 = correlation_matrix.columns[j]
            correlation = correlation_matrix.iloc[i, j]
            correlation_pairs.append((feature1, feature2, correlation))

for feature1, feature2, correlation in correlation_pairs:
    print(f"Correlation between {feature1} and {feature2}: {correlation:.2f}")

 To avoid collinearity issues, the numerical variables *HeartRate_Mean*, *MeanBP_Mean*, *TempC_Mean* and *Glucose_Mean* are dropped from both the training and testing set:

In [ ]:
X_train = X_train.drop(['HeartRate_Mean', 'MeanBP_Mean', 'TempC_Mean', 'Glucose_Mean'], axis = 1)
X_test = X_test.drop(['HeartRate_Mean', 'MeanBP_Mean', 'TempC_Mean', 'Glucose_Mean'], axis = 1)

In [ ]:
X_train.head()

## Encode categorical data

### Low cardinality categorical features

The low cardinality categorical variables will be encoded using one-hot encoding to transform categorical variables into dummies. The parameter *drop_first = True* is used to get k-1 dummies out of k categorical levels by removing the first level.

In [ ]:
columns_to_encode = ['GENDER', 'INSURANCE', 'ADMISSION_TYPE', 'MARITAL_STATUS', 'FIRST_CAREUNIT']

for col in columns_to_encode:
    X_train = pd.get_dummies(X_train, prefix=[col], columns=[col], drop_first=True)
    X_test = pd.get_dummies(X_test, prefix=[col], columns=[col], drop_first=True)

X_test.head()

### High cardinality categorical features

For high cardinality categorical variables target encoder is used with *TargetEncoder* from the *Category Encoders* package. According to the documentation from *TargetEncoder*, for the case of a categorical target (in our case 'HOSPITAL_EXPIRE_FLAG'), features are replaced with a blend of posterior probability of the target given particular categorical value and the prior probability of the target over all the training data (Source: https://contrib.scikit-learn.org/category_encoders/targetencoder.html). *Smoothing* corresponds to the smoothing effect to balance categorical average versus the prior. This parameter is set to 10 because if the value is increased a lot then it contributes to overfitting and if it is more closest to 0 (since it should be strictly larger than 0) then the model looses predictive accuracy. The target encoder is fitted with training data and applied to both training and testing dataset. 

In [ ]:
columns = ['ICD9_diagnosis', 'DIAGNOSIS', 'ICD9_CODE_2', 'ICD9_CODE_3', 'ICD9_CODE_4', 'ICD9_CODE_5', 'ICD9_CODE_6', 'ICD9_CODE_7', 'ICD9_CODE_8', 'ICD9_CODE_9', 'ICD9_CODE_10', 'ICD9_CODE_11', 'ICD9_CODE_12']

for col in columns:
    te = TargetEncoder(smoothing = 10)
    te.fit(X = X_train[col], y = y_train)
    values_train = te.transform(X_train[col])
    values_test = te.transform(X_test[col])
    X_train[col] = values_train
    X_test[col] = values_test

X_train[columns].head()

## Transform metadata columns

The addition of secondary codes increase the complexity of the model, therefore it seems reasonable to convert those columns into summary statistics and drop the initial columns from metadata. For example, including the mean can be useful to capture the central tendency of comorbidities for each patient.

In [ ]:
columns_to_convert_statistics = ['ICD9_CODE_2', 'ICD9_CODE_3', 'ICD9_CODE_4', 'ICD9_CODE_5', 'ICD9_CODE_6', 'ICD9_CODE_7', 'ICD9_CODE_8', 'ICD9_CODE_9', 'ICD9_CODE_10', 'ICD9_CODE_11', 'ICD9_CODE_12']
    
X_train['METADATA_MEAN'] = X_train[columns_to_convert_statistics].mean(axis=1)
X_train['METADATA_MAX'] = X_train[columns_to_convert_statistics].max(axis=1)

X_train = X_train.drop(columns_to_convert_statistics, axis=1)

X_test['METADATA_MEAN'] = X_test[columns_to_convert_statistics].mean(axis=1)
X_test['METADATA_MAX'] = X_test[columns_to_convert_statistics].max(axis=1)

X_test = X_test.drop(columns_to_convert_statistics, axis=1)

It can be seen that the dimensions from both the training and testing dataset had been reduced by 9 columns.

In [ ]:
X_train.head()

## Feature standardization

Features are scaled using *StandardScaler*. The parameters *with_mean* and *with_std* are set to *True* which implies that data is centered before scaling and it scale the data to unit variance. The scaler is fitted with the training dataset and then applied to both training and testing dataset.

In [ ]:
scaler = preprocessing.StandardScaler(with_mean = True, with_std = True)
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train) 
X_test = pd.DataFrame(X_test)

# KNN model

First, it is created an instance of a K-nearest neighbors classifier (KNeighborsClassifier) and set the *algorithm* parameter to *'brute'* which involves the brute-force computation of distances between all pairs of observations in the dataset. Performing grid search over *algorithm* is used to make the algorithm faster not to improve performance, that is why it is not included in the GridSearch.

The hyperparameter optimization is done before fitting the model. The parameters tuned in the GridSearch corresponds to:
- *n_neighbors* which is the number of neighbors. This parameter is grid search over a range from 50 to 350 with increases of 50 since a low number of neighbors increases variance but a high number increases bias.
- *weights* which refers to the weight function used in prediction. While *uniform* distance uses uniform weights, *distance* weights observations by the inverse of the distance.
- *metric* which corresponds to the distance measure.

Then the SVM model is fitted with the *X_train* and *y_train*.

In [ ]:
MyKNN = KNeighborsClassifier(algorithm =  'brute')

grid_values = {'n_neighbors':[100, 150, 200, 250, 300, 350, 400, 450], 'weights':['uniform','distance'], 'metric': ['manhattan', 'minkowski']}

grid_knn_acc = GridSearchCV(MyKNN, param_grid = grid_values, scoring = 'roc_auc', n_jobs = -1, cv = 20) # CV defines the number of folds 

grid_knn_acc.fit(X_train, y_train)

## Optimal parameters from the grid

The optimal parameters from the grid correspond to 350 neighbors, the optimal *weights* is *'distance'* and the *metric* is *'manhattan'*. The resulting best score equals 0.93683. 

In [ ]:
GridSearch_table_plot(grid_knn_acc, "n_neighbors", negative=False, display_all_params=False)

print('Best k parameter : '+ str(grid_knn_acc.best_estimator_.n_neighbors))
print('Best weights parameter : '+ str(grid_knn_acc.best_estimator_.weights))
print('Best metric parameter : '+ str(grid_knn_acc.best_estimator_.metric))

## Out-of-sample predictions

The out-of-sample predictions uses the data from the *X_test* which has not been used to train the model.

In [ ]:
knn_outsample_pred_prob = grid_knn_acc.predict_proba(X_test)

## In-sample predictions

The in-sample predictions uses the data from the *X_train* which has been used to train the model.

In [ ]:
knn_insample_pred_prob = grid_knn_acc.predict_proba(X_train)

print("AUC: ", roc_auc_score(y_train, knn_insample_pred_prob[:, 1]))

The ROC AUC score obtained from the in-sample prediction corresponds to 1. Since the score is 1, it suggests that the model perfectly distinguish between patients who die or survive. Through the following confusion matrix it can be seen that all the observations from the training dataset are correctly classified, thus obtaining a False Positive Rate and False Negative Rate of 0. Said result can also suggest overfitting, which may be addressed as a future line to improve the project. However, the optimal parameters have been selected using cross-validation, which mitigates the effect of overfitting, that is why the best score equals 0.93683.

In [ ]:
print("Confusion matrix")
cm=confusion_matrix(y_train,knn_insample_pred_prob[:,1])
plot_confusion_matrix(cm, ['Survived','Not survived'])

# Output predictions file

The following code produce a csv file with the output predictions to submit them to Kaggle:

In [ ]:
test_predictions_submit_knn = pd.DataFrame({"icustay_id": data_test["icustay_id"], "HOSPITAL_EXPIRE_FLAG": knn_outsample_pred_prob[:, 1]})
test_predictions_submit_knn.to_csv("test_predictions_submit_knn.csv", index = False)

The file containing the output predictions has the following form:

In [ ]:
test_predictions_submit_knn.head()